<div id="titre">
<h1 style="font-size:500%">Data visualisation des séismes</h1>
<div style="font-family:Comic Sans MS; background-color:black;box-shadow: 8px 8px 0px #aaa; text-align:center; width: 50%; color:white">Bro Frédéric - PFC - Liban - 20 janvier 2020</div>
</div>

L'USGS fournit des données sur les tremblements de terre survenus dans le monde à l'adresse https://earthquake.usgs.gov/earthquakes/search/ . 
<br> Le fichier <code>'earthquakes.csv'</code> contient les données concernant tous les séimes qui se sont produits en 2014.

<div id="table" style="border:solid; background-color:#86BC98;box-shadow: 8px 8px 0px #aaa">
    <p style="font-family:Comic Sans MS"><u>Table des matières :</u></p>
<p style="font-family:Arial Black;font-size:100%; color:black">1 - <a href="#section1" style="font-family:Comic Sans MS">Quels sont les pays les plus touchés par les séismes ?</a></p>
<p style="font-family:Arial Black; font-size:100%; color:black">2 - <a href="#section2" style="font-family:Comic Sans MS">Création de cartes des séismes produits en 2014</a></p>
<p style="font-family:Arial Black; font-size:100%; color:black">3 - <a href="#section3" style="font-family:Comic Sans MS">Carte choroplèthe associée au nombre de séismes</a></p>
</div>

<div id="section1">
<blockquote id="section">Quels sont les pays les plus touchés par les séismes ?</blockquote>
<a href="#table"; style="text-align: right;">Retour table des matières</a>
</div>

In [ ]:
import pandas as pa 
import pylab as pl
import seaborn
pl.style.use('bmh')

In [ ]:
T = pa.read_csv('earthquakes_2014.csv')
T.head(2)

<div class="alert alert-success" role="alert">
    <p style="font-family:Arial Black">Récupérons le nom du pays auquel s'est produit le séisme :</p>
    <blockquote id="barrev">
        <ul>
            <li>La colonne <code>place</code> renseigne le lieux du séisme. 
            <br>Le nom situé juste après la virgule correspond au nom du pays où s'est produit le séisme.</li>
            <li>Pour chaque séime, on récupère le nom du pays où s'est déroulé le séisme et on stocke ces noms dans la nouvelle colonne <code>pays</code> de <code>T</code></li>
        </ul>
    </blockquote>
</div>

In [ ]:
T['pays'] = T.place.str.split(',').apply(lambda x:x[-1])
T['pays'] = T.pays.str.replace(' ','')

T['pays'].value_counts().head(20)

In [ ]:
noms = T['pays'].value_counts().head(20).index

pl.figure(figsize=(10,4))
pl.xticks(rotation=90)
seaborn.boxplot(x='pays', y='mag', data=T.query('pays in @noms'),
                whis='range', order=noms)
pl.show()

<div class="alert alert-success" role="alert">
    <p style="font-family:Arial Black">Remarque :</p>
    <blockquote id="barrev">
        La force de pandas permet en quelques instants de nous apporter les données.
    </blockquote>
</div>

<div id="section2">
<blockquote id="section">Création de cartes des séismes produits en 2014</blockquote>
<a href="#table"; style="text-align: right;">Retour table des matières</a>
</div>

<blockquote id="barre">
    <ul>
        <li> Le module geopandas permet de lire le fichier <code>'countries.json'</code>.</li>
        <li> Ce fichier contient les coordonnées gps des frontières de chaque pays du monde </li>
    </ul>
</blockquote>

In [ ]:
import geopandas as gp

monde = gp.read_file('countries.json')
monde.head()

<div class="alert alert-info">
    <p style="font-family:Arial Black">Objectif :</p> 
    <blockquote id="barre">
    Sélectionnons les seismes de magnitude $\geqslant 1$ et représentons les sur une carte selon sa magnétude.
    </blockquote>    
</div>

In [ ]:
F = T.query('mag >= 1')

fig, ax = pl.subplots(figsize=(15, 8))
monde.plot(color='darksalmon',ax=ax)
pl.scatter(F['longitude'],F['latitude'],c=F['mag'],cmap='coolwarm')

ax_cbar = pl.colorbar(fraction=0.1,pad=0.05,shrink=0.7)
ax_cbar.set_label('Magnitude')

In [ ]:
len(F)

<div class="alert alert-success" role="alert">
<p style="font-family:Arial Black">Objectif :</p>
<blockquote id="barrev">
Les $82456$ séismes se répartissent le long des plaques tectoniques.
</blockquote>
</div>

<div class="alert alert-info">
    <p style="font-family:Arial Black">Objectif :</p> 
    <blockquote id="barre">
        Créons une carte "dynamique" représentant les $36203$ séismes de <code>F</code>.
    </blockquote>    
</div>

<blockquote id="barre">
    <ul>
        <li> Le module folium permet de représenter des cartes dynamique <code>'countries.json'</code>.</li>
        <li> Nous allons utiliser les cartes proposées par <b>Open Street Map</b> et le langage Python afin de générer des cartes personnalisées. Plus exactement, nous allons utiliser une bibliothèque Python nommée <code>Folium</code>, pour créer nos propres cartes à partir des cartes proposées par <b>Open Street Map</b> </li>
    </ul>
</blockquote>

In [ ]:
import folium

carte = folium.Map(location=[0,0],tiles='stamenterrain', zoom_start=2)
carte

In [ ]:
from folium.plugins import HeatMap

F = T.query('mag > 2')

GPS = F[['latitude','longitude']].values.tolist()

HeatMap(GPS,radius=4,blur=1,gradient={0.5: 'dark', 0.5: 'blue', 1: 'red'},
        min_opacity=0.4).add_to(carte)
carte

In [ ]:
len(F)

<div class="alert alert-info">
    <p style="font-family:Arial Black">Objectif :</p> 
    <blockquote id="barre">
        Représentons par dessus <code>carte</code> les sismes de magnétude $\geqslant 6$.
    </blockquote>    
</div>

In [ ]:
F = T.query('mag >= 6')

from math import sqrt

for i,ligne in F.iterrows():
    folium.CircleMarker(location=(ligne['latitude'],ligne['longitude']),
                        popup=ligne['pays']+'\n'+str(ligne['mag']),
                        color='green',
                        fill_color='yellow',
                        radius=5*sqrt(ligne['mag'])).add_to(carte)
carte

In [ ]:
len(F)

<div id="section3">
<blockquote id="section">Carte choroplèthe associée au nombre de séismes</blockquote>
<a href="#table"; style="text-align: right;">Retour table des matières</a>
</div>

<blockquote id="barre">
        Créons la table <code>E</code> des effectifs des séismes qui se sont produits 
</blockquote> 

In [ ]:
E = T.groupby('pays').count().time.reset_index().query('time >= 10')
E.head()

In [ ]:
monde.head()

<div class="alert alert-info">
    <p style="font-family:Arial Black">Objectif :</p> 
    <blockquote id="barre">
        Fusionnons la table <code>monde</code> avec la table <code>E</code> à partir de :
        <ul>
            <li>la colonne <code>ADMIN</code> de <code>monde</code>.</li>
            <li>et la colonne <code>pays</code> de <code>E</code>.</li>
        </ul>
    </blockquote>    
</div>

In [ ]:
M = monde.merge(E,left_on='ADMIN',right_on='pays',how='inner')
M.head()

<div class="alert alert-info">
    <p style="font-family:Arial Black">Objectif :</p> 
    <blockquote id="barre">
        Créons la carte des pays du monde en coloriant chacun d'eux avec une couleur selon le nombre de séismes produits dans le pays.
    </blockquote>    
</div>

In [ ]:
M.crs = {'init' :'epsg:robin'}

fig, ax = pl.subplots(figsize=(16, 13))
monde.plot(ax=ax,edgecolor='black',color='white')
M.plot(ax=ax,column='time', cmap='coolwarm',edgecolor='black',
       alpha=1,scheme='Quantiles')


"""Représentation de la palette de couleur"""
vmin = M['time'].min()
vmax = M['time'].max()
norm=pl.Normalize(vmin=vmin, vmax=vmax)

cax = fig.add_axes([0.93, 0.3, 0.03, 0.4])
sm=pl.cm.ScalarMappable(cmap='coolwarm', norm=norm)
sm._A = []
fig.colorbar(sm, cax=cax,orientation='vertical')

<p style="color:blue;font-family:Comic Sans MS;background-color:white;font-size:200%;text-align:center;border-style:solid solid solid solid;padding:10px">Fini ... </p>
<p>
<a href="#titre"; style="text-align: right;">Retour au début</a>
</p>

In [1]:
from IPython.core.display import HTML
htm = open("style_css.txt").read()
HTML(htm)